In [1]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [2]:
df_movie_details = pd.read_json('../data/IMDB_movie_details.json', lines = True)
df_reviews = pd.read_pickle('../data/cleaned_reviews.pkl.gz', compression = 'gzip')

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df_reviews.loc[:, 'tokenized_reviews'], df_reviews.loc[:, 'is_spoiler'], test_size = 0.2)

In [5]:
model = Word2Vec(X_train, vector_size = 1000)

In [6]:
model.train(X_train, total_examples = model.corpus_count, epochs = 1)

(59443609, 64957681)

In [7]:
model.wv.most_similar(X_test)

ValueError: operands could not be broadcast together with shapes (114783,) (0,) 